In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [ ]:
import torch
from torchvision import datasets, models, transforms
import os
import glob
from PIL import Image
import numpy as np
import json
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
basePath = '/content/'
#data_dir = basePath+'Sample'
data_dir = basePath+'val2017'

topk = 44
path = data_dir+'/135/135.png'

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
'''drive_dir = '/content/drive/My Drive/RE/'
import zipfile
import os
for file_name in os.listdir(drive_dir):
  if file_name.endswith('.zip'):
    with zipfile.ZipFile(drive_dir+file_name,'r') as zip_dir:
      zip_dir.extractall(path='/content/')'''

"drive_dir = '/content/drive/My Drive/RE/'\nimport zipfile\nimport os\nfor file_name in os.listdir(drive_dir):\n  if file_name.endswith('.zip'):\n    with zipfile.ZipFile(drive_dir+file_name,'r') as zip_dir:\n      zip_dir.extractall(path='/content/')"

In [ ]:
input_size = 224
def resize_and_rename(image_dir):
    i = 1
    files = glob.glob(os.path.join(image_dir, "*"))
    for file in files:
        im = Image.open(file)
        f, e = os.path.splitext(file)
        imResize = im.resize((input_size,input_size), Image.ANTIALIAS)
        os.remove(file)

        path = '/'+os.path.join(*f.split('/')[:-1])+'/'+str(i)
        if not os.path.exists(path):
          os.mkdir(path) 
        fullPath = os.path.join(path,str(i))
        i+=1
        imResize.save(fullPath + '.png', 'PNG', quality=90)

In [ ]:
'''!cp -R '/content/drive/My Drive/Sample' '/content/'
#!cp -R '/content/drive/My Drive/COVID-19/Dataset_2' '/content/'
resize_and_rename(data_dir)'''

"!cp -R '/content/drive/My Drive/Sample' '/content/'\n#!cp -R '/content/drive/My Drive/COVID-19/Dataset_2' '/content/'\nresize_and_rename(data_dir)"

In [ ]:
def ReLU(X):
   return np.maximum(0,X)

In [ ]:
def CReLU(f):
    f_pos = f
    f_neg = -f
    final_f = np.concatenate((f_pos, f_neg))
    return ReLU(final_f)

In [ ]:
def getSurrogateText(F,topK):
    surrText = ""
    count = topK
    for ele in F:
        instanceText = "RO"+str(ele)
        instanceText += (" "+instanceText)*(count-1)
        if len(surrText) == 0:
            surrText = instanceText
        else:
            surrText += (" "+instanceText)
        count -= 1
    return surrText

In [ ]:
def F2Text(f,topK):
    rf = CReLU(f)
    perVec = np.argsort(rf)[::-1]+1
    invPer = np.argsort(perVec)+1
    after_topk = invPer[invPer <= topK]
    return getSurrogateText(after_topk,topK)

In [ ]:
class DenseFeatures(nn.Module):
    def __init__(self,dense):
        super(DenseFeatures, self).__init__()
        self.features = nn.Sequential(*list(dense.children())[:-1])
        #self.feat_vec = nn.Sequential(*list(dense.children())[2][:-3])
        
    def forward(self, x):
        x = self.features(x)
        x = F.relu(x, inplace=True)
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = torch.flatten(x, 1)
        #x = torch.flatten(x, 1)
        #x= self.feat_vec(x)
        return x

dense = models.densenet121(pretrained=True)
model = DenseFeatures(dense)
model.eval()
model = model.to(device)

In [ ]:
data_transforms = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

In [ ]:
path = '/content/val2017/135/135.png'
image = Image.open(path)
x = data_transforms(image)
x.unsqueeze_(0)
x=x.to(device)
print(x.shape)

torch.Size([1, 3, 224, 224])


In [ ]:
#print(path)
#print(x)
output = model(x).cpu().detach().numpy()
#print(output)
surrText = F2Text(output[0],topk)
print(surrText)

RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO25 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO35 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO37 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO43 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 RO38 